# Visualizing a Video Dataset #

This notebook is a simple demonstration of how to use vgrid to visualize a video dataset.  The tutorial assumes that the video files, as well as precomputed metadata about the videos, are located on a remote machine. I expect this to be a common case in a lot of workflows.

In [ ]:
import urllib3, requests, json, os

# HACK(kayvonf): I don't want to see certificate warnings from olimar.stanford.edu
urllib3.disable_warnings()

In [ ]:
# location of the video metadata file.
# It is assumed that video data is located relative to this file.
VIDEO_COLLECTION_BASEURL = "http://olimar.stanford.edu/hdd/kayvon10/" 
VIDEO_METADATA_FILENAME = "kayvon10.json"

In [ ]:
#
# The first step in the process is to grab the metadata about my video collection.
#
req = requests.get(os.path.join(VIDEO_COLLECTION_BASEURL, VIDEO_METADATA_FILENAME), verify=False)
video_collection = req.json()
print("The video collection has %d videos." % len(video_collection))

In [ ]:
from rekall import Interval, IntervalSet, IntervalSetMapping, Bounds3D
from vgrid.vblocks_builder import VideoVBlocksBuilder, VideoTrackBuilder, DrawType_Bbox, VideoMetadata
from vgrid_jupyter import VGridWidget

video_metadata = []

# HACK(kayvonf): At the moment, VideoVBlocksBuilder doesn't seem to build anything
# if at least one track is not specified. I'm providing a dummy track here to keep it
# happy for now. This needs to be fixed.
test_intervals = {}

for v in video_collection:
    video_metadata.append(VideoMetadata(v["filename"], v["id"], v["fps"], v["num_frames"], v["width"], v["height"]))
    test_intervals[v["id"]] = IntervalSet([Interval(Bounds3D(0, 1)), Interval(Bounds3D(2, 3))])
    
builder = VideoVBlocksBuilder()
builder.add_video_metadata(VIDEO_COLLECTION_BASEURL, video_metadata)
builder.add_track(VideoTrackBuilder('test', IntervalSetMapping(test_intervals)))

# now make the vgrid widget
VGridWidget(vgrid_json=builder.build())